In [1]:
import numpy as np
import pandas as pd

In [2]:
x_train= pd.read_csv('data_reviews/x_train.csv')
y_train= pd.read_csv('data_reviews/y_train.csv')

In [3]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

In [4]:
import math

In [5]:
import re
import nltk

In [6]:
#lower case
x_train['text_processed'] = x_train['text'].map(lambda x: x.lower())

# contractions
def contractions(s):
    s = re.sub(r"won’t", "will not",s)
    s = re.sub(r"would’t","would not",s)
    s = re.sub(r"could’t", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
x_train['text_processed'] = x_train['text_processed'].map(lambda x: contractions(x))

x_train['text_processed'] = x_train['text_processed'].map(lambda x: re.sub('[.!,\.?()0123456789:;\x85\x96\x97"#$%&()\[\]*+-/\']', '', x))

# remove non-alpha words
import nltk
x_train['text_processed'] = x_train['text_processed'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','',x) for x in nltk.word_tokenize(x)]))

# remove extra spaces
x_train['text_processed'] = x_train['text_processed'].apply(lambda x: re.sub(' +',' ', x))

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
x_train['1'] = x_train['text_processed'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
x_train['1'] = x_train['1'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
x_train['2'] = x_train['text_processed'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ('mlp', MLPClassifier(alpha = 7.943282347242816, hidden_layer_sizes = (70,), max_iter = 50, solver = 'lbfgs'))
    ]
)
scores = cross_validate(pipeline, x_train['text_processed'],y_train.values.reshape(-1))
print(scores)


C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: Conv

{'fit_time': array([3.26399207, 3.00500464, 2.6303196 , 2.85493112, 2.94440722]), 'score_time': array([0.        , 0.01561904, 0.01561689, 0.        , 0.        ]), 'test_score': array([0.82916667, 0.825     , 0.7625    , 0.78958333, 0.83958333])}


C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [22]:
np.mean(scores['test_score'])

0.8091666666666667

In [28]:
pipeline.fit(x_train['text_processed'],y_train.values.reshape(-1))

C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Pipeline(steps=[('vect', CountVectorizer()),
                ('mlp',
                 MLPClassifier(alpha=7.943282347242816,
                               hidden_layer_sizes=(70,), max_iter=50,
                               solver='lbfgs'))])

In [35]:
pipeline.score(x_train['text_processed'],y_train.values.reshape(-1))

0.9958333333333333

In [33]:
from sklearn.metrics import confusion_matrix
y_pred = pipeline.predict(x_train['text_processed'])
confusion_matrix(y_train, y_pred)

array([[1199,    1],
       [   9, 1191]], dtype=int64)

In [38]:
from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
pipeline.fit(xtr['text_processed'],ytr.values.reshape(-1))

C:\Users\Zhaohui Zhang\.conda\envs\ml135_env_sp21\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Pipeline(steps=[('vect', CountVectorizer()),
                ('mlp',
                 MLPClassifier(alpha=7.943282347242816,
                               hidden_layer_sizes=(70,), max_iter=50,
                               solver='lbfgs'))])

In [43]:
y_pred = pipeline.predict(xte['text_processed'])
confusion_matrix(yte, y_pred)

array([[198,  39],
       [ 35, 208]], dtype=int64)

In [44]:
1-(39+35)/(2400*0.2)

0.8458333333333333

In [66]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

incorrects = np.nonzero(pipeline.predict(xte['text_processed']).reshape(-1) != yte.values.reshape(-1))
xte.iloc[incorrects].join(yte.iloc[incorrects])

C:\Users\ZHAOHU~1\AppData\Local\Temp/ipykernel_16968/2033136125.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,website_name,text,text_processed,1,2,is_positive_sentiment
1289,imdb,It was clear that she had the range and ability to pull off this part.,it was clear that she had the range and ability to pull off this part,clear range ability pull part,it wa clear that she had the range and ability to pull off this part,1
111,amazon,Disapointing Results.,disapointing results,disapointing result,disapointing result,0
1198,imdb,"The results, well, are a shame.",the results well are a shame,result well shame,the result well are a shame,0
1907,yelp,"for 40 bucks a head, i really expect better food.",for bucks a head i really expect better food,buck head really expect better food,for buck a head i really expect better food,0
1521,imdb,All in all I give this one a resounding 9 out of 10.,all in all i give this one a resounding out of,give one resounding,all in all i give this one a resounding out of,1
1565,imdb,"Highly recommended for all ages, although the younger set will probably not appreciate some of the more subtle references, they will certainly appreciate one galley scene in particular!",highly recommended for all ages although the younger set will probably not appreciate some of the more subtle references they will certainly appreciate one galley scene in particular,highly recommended age although younger set probably appreciate subtle reference certainly appreciate one galley scene particular,highly recommended for all age although the younger set will probably not appreciate some of the more subtle reference they will certainly appreciate one galley scene in particular,1
1301,imdb,I advise you to look out for it.,i advise you to look out for it,advise look,i advise you to look out for it,1
2054,yelp,Def coming back to bowl next time,def coming back to bowl next time,def coming back bowl next time,def coming back to bowl next time,1
2024,yelp,This place has it!,this place has it,place,this place ha it,1
218,amazon,Kind of flops around.,kind of flops around,kind flop around,kind of flop around,0


In [73]:
x_train= pd.read_csv('data_reviews/x_test.csv')  #save as train so no need change code


#lower case
x_train['text_processed'] = x_train['text'].map(lambda x: x.lower())

# contractions
def contractions(s):
    s = re.sub(r"won’t", "will not",s)
    s = re.sub(r"would’t","would not",s)
    s = re.sub(r"could’t", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
x_train['text_processed'] = x_train['text_processed'].map(lambda x: contractions(x))

x_train['text_processed'] = x_train['text_processed'].map(lambda x: re.sub('[.!,\.?()0123456789:;\x85\x96\x97"#$%&()\[\]*+-/\']', '', x))

# remove non-alpha words
import nltk
x_train['text_processed'] = x_train['text_processed'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','',x) for x in nltk.word_tokenize(x)]))

# remove extra spaces
x_train['text_processed'] = x_train['text_processed'].apply(lambda x: re.sub(' +',' ', x))

yproba1_test = pipeline.predict_proba(x_train['text_processed'])[:, 1]
np.savetxt('yproba1_test.txt', yproba1_test)